In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import csv
import json
import csv
import math
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.simplefilter("ignore")
import joblib
import pickle
import nltk 
nltk.download('punkt')
import tensorflow as tf 
from tensorflow.keras.layers import Dense
import random
from nltk.stem.lancaster import LancasterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def get_precaution(text): 
  disease = get_disease(text)
  data = pd.read_csv('drive/MyDrive/disymp/symptom_precaution.csv')
  data['Disease'] = data['Disease'].apply(lambda x: x.lower())
  data['Disease'] = data['Disease'].apply(lambda x:x.strip())
  precaution = data[data['Disease'] == disease].values
  prec=[]
  if(precaution.shape[0] != 0):
    for i in range(1,precaution.shape[1]):
      if type(precaution[0][i]) == str:
        prec.append(precaution[0][i])
  return prec

In [ ]:
def get_description(text):
  disease = get_disease(text)
  data = pd.read_csv('drive/MyDrive/disymp/symptom_Description.csv')
  data['Disease'] = data['Disease'].apply(lambda x: x.lower())
  data['Disease'] = data['Disease'].apply(lambda x:x.strip())
  description = data[data['Disease'] == disease].values
  desc = None
  if description.shape[0]!=0:
    desc = description[0][1]
  return desc

In [ ]:
def get_symptoms(text): 
  symptoms = pd.read_csv('drive/MyDrive/disymp/symptoms.csv')
  symp_list = list(symptoms['Symptoms'].values)
  symps = []
  for i in symp_list: 
    if(text.find(i)!=-1):
      symps.append(i)
  return symps

In [ ]:
def get_intent(text):

  with open('drive/MyDrive/disymp/intentcla.pkl', 'rb') as f:
    count_vect, clf = pickle.load(f)  
  intent = clf.predict(count_vect.transform([text]))
  return str(intent).strip("['']")

In [ ]:
def get_sympvector(text):
  symps = get_symptoms(text)
  sympid = pd.read_csv('drive/MyDrive/disymp/symptoms.csv')
  sympid.columns = ['idx','Symptoms']
  sympid.set_index('Symptoms')
  indexes = []
  for i in symps:
    indexes.append(sympid[sympid['Symptoms'] == i]['idx'].values[0])
  vec = [0]*131
  for i in range(len(indexes)): 
    vec[indexes[i]] = 1
  return np.array(vec)

In [ ]:
def classify_disease(text):
  vec = get_sympvector(text).reshape(1,131)
  classifier = joblib.load('drive/MyDrive/disymp/discla.pkl')
  return classifier.predict(vec)[0]

In [ ]:
def get_disease(text): 
  dis = pd.read_csv('drive/MyDrive/disymp/disease.csv')
  dislist = list(dis['Diseases'].values)
  disease = ''
  for i in dislist:
    if text.find(i)!=-1:
      disease = i
      break
  return disease

In [ ]:
def get_response(text):
  stemmer = LancasterStemmer()
  with open('drive/MyDrive/disymp/diaintents.json') as json_data: 
    intents = json.load(json_data)
  words = []
  classes = []
  documents = []
  ignore_words = ['?']
  for intent in intents['intents']:
      for pattern in intent['patterns']:

          w = nltk.word_tokenize(pattern)

          words.extend(w)

          documents.append((w, intent['tag']))

          if intent['tag'] not in classes:
              classes.append(intent['tag'])

  words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
  words = sorted(list(set(words)))

  classes = sorted(list(set(classes)))

  def clean_up_sentence(sentence):

      sentence_words = nltk.word_tokenize(sentence)

      sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
      return sentence_words

  def bow(sentence, words, show_details=False):

      sentence_words = clean_up_sentence(sentence)

      bag = [0]*len(words)  
      for s in sentence_words:
          for i,w in enumerate(words):
              if w == s: 
                  bag[i] = 1
                  if show_details:
                      print ("found in bag: %s" % w)

      return(np.array(bag))
  context = {}

  ERROR_THRESHOLD = 0.25
  def classify(sentence):
      model = tf.keras.models.load_model('drive/MyDrive/disymp/diamodel.h5')
      results = model.predict([bow(sentence, words).reshape(1,47)])[0]
      results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
      results.sort(key=lambda x: x[1], reverse=True)
      return_list = []
      for r in results:
          return_list.append((classes[r[0]], r[1]))
      return return_list

  def response(sentence, userID='123', show_details=False):
      results = classify(sentence)

      if results:

          while results:
              for i in intents['intents']:

                  if i['tag'] == results[0][0]:

                      if 'context_set' in i:
                          if show_details: print ('context:', i['context_set'])
                          context[userID] = i['context_set']

                      if not 'context_filter' in i or \
                          (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                          if show_details: print ('tag:', i['tag'])
                          return random.choice(i['responses'])

              results.pop(0)
  return response(text) 

In [ ]:
while True: 
  user_text = input('User: ')
  user_text = user_text.lower()
  intent = get_intent(user_text)
  if(user_text == 'hello'):
    print("Chatbot: hello")
  elif user_text=='exit':
    break 
  elif intent == 'disease_descritpion':
    print('Chatbot: ',get_description(user_text))
  elif intent == 'disease_precaution': 
    reply = get_precaution(user_text)
    for i in reply:
      print('Chatbot: ',i)
  elif intent == "symptoms_checker":
    print("Chatbot: You are diagonsed with: ",classify_disease(user_text))
    if(classify_disease(user_text) == 'Diabetes '):
      print('Chatbot: Please enter query if you would like to know more about diabetes or else type exit: ')
      text = input("User: ")
      if text == 'exit':
        break
      else: 
        print('Chatbot: ',get_response(text))

User: hello
Chatbot: hello
User: exit
